In [1]:
import re
import dill
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
import seaborn as sns
sns.set()

# Scrape race pages

In [17]:
race_ids = list(range(1, 13000))
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/race/{i}') for i in race_ids]

In [18]:
for i, future in enumerate(futures):
    dill.dump(future.result().text, open(f'C:/data/results/{i}.pkd', 'wb'))

## Extract race metadata

In [20]:
def get_data(text):
    name = ''
    date = ''
    loc = ''
    json_url = ''
    weather = ''
    strava_url = ''
    coord = None
    
    if 'No data' not in text:
        soup = BeautifulSoup(text)
        resultstitle = soup.select('div#resultstitle')[0].text
        split = resultstitle.split('•')

        regex = re.compile(r'([A-Za-z]{3})\s+(\d{1,2})\s+(\d{4})')
        name = split[0].strip()
        month, day, year = regex.findall(split[1].strip())[0]
        date = datetime.strptime(' '.join([month, day, year]), '%b %d %Y')
        loc = ''
        if len(split) > 2:
            loc = re.split('[\t\n\r]', split[2])[0].strip()
            
        json_url = soup.select('span.downloadoptions')[0].find_all('a')[1]['href']
        weather = soup.select('div#conditionssummary')[0].text.strip()
        strava_url = soup.select('div#stravasegment')[0].select('a')[1]['href']
        
        try:
            coord = re.findall(r'GetMap\("([-\d\.]+):([-\d\.]+)"', text)[0]
        except IndexError:
            pass
            
    return name, date, loc, json_url, weather, strava_url, coord

In [21]:
columns=['name', 'date', 'loc', 'json_url', 'weather', 'strava_url', 'coord']
df = pd.DataFrame(columns=columns)
for i in race_ids:  # loop takes ~2 hour or so
    if i % 100 == 0:
        print(i)
    if i in [12533, 12534]:  # these broke for some reason >:|
        continue
    text = dill.load(open(f'C:/data/results/{i}.pkd', 'rb'))
    data = get_data(text)
    df.loc[i] = {c: d for c, d in zip(columns, data)}

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900


FileNotFoundError: [Errno 2] No such file or directory: 'C:/data/results/12999.pkd'

In [22]:
df.to_pickle('C:/data/results/df.pkl')

## reload, fix some things, and resave

In [111]:
df = pd.read_pickle('C:/data/results/df.pkl')

In [146]:
df = df[df['json_url'] != '']  # drop if no results
df['weather'] = df['weather'].str.split('\n').str.get(-1)  # sometimes the weather column also got "Series: " information
df.index = df['json_url'].apply(lambda x: int(re.findall('(\d+)', x)[0]))  # find digits in url
df.index.name = 'race_id'

In [147]:
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
race_id,,,,,,,
986,Sublime Sublimity,2011-06-11 00:00:00,,downloadrace.php?raceID=986&json=1,,http://app.strava.com/segments/,None
11222,21th Annual Paragon Hungry Dog Criterium,2019-03-10 00:00:00,"Gilbert, AZ",downloadrace.php?raceID=11222&json=1,"Clear, 67 degrees, wind 3 mph",http://app.strava.com/segments/,"(33.330509185791, -111.790260314941)"
12351,Driveway Series,2019-07-04 00:00:00,"Austin, TX",downloadrace.php?raceID=12351&json=1,"Humid and Mostly Cloudy, 86 degrees, wind 12 mph",http://app.strava.com/segments/,"(30.2675895690918, -97.7429885864258)"
5846,Nevada Cycling Road Race,2015-04-11 00:00:00,"Reno, NV",downloadrace.php?raceID=5846&json=1,"Partly Cloudy, 56 degrees, wind 7 mph",http://app.strava.com/segments/,None
8437,LSU Criterium,2017-02-26 00:00:00,"St. Francisville, LA",downloadrace.php?raceID=8437&json=1,"Clear, 63 degrees, wind 10 mph",http://app.strava.com/segments/,None


In [148]:
df.to_pickle('C:/data/results/df.pkl')

## reload

In [149]:
df = pd.read_pickle('C:/data/results/df.pkl')
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
race_id,,,,,,,
5740,Naval Academy Criterium,2015-03-29 00:00:00,"Annapolis, MD",downloadrace.php?raceID=5740&json=1,"Clear, 37 degrees, wind 8 mph",http://app.strava.com/segments/,None
10788,Colorado Classic - Stage 4,2018-08-19 00:00:00,"Denver, CO",downloadrace.php?raceID=10788&json=1,"Mostly Cloudy, 66 degrees, wind 2 mph",http://app.strava.com/segments/,None
11490,Spin the District College Park Criterium,2019-05-05 00:00:00,"College Park, GA",downloadrace.php?raceID=11490&json=1,"Overcast, 70 degrees, wind 4 mph",http://app.strava.com/segments/,"(33.6564292907715, -84.4487762451172)"
3927,Ontario Series 3,2014-03-23 00:00:00,"Ontario, CA",downloadrace.php?raceID=3927&json=1,"Clear, 64 degrees, wind 6 mph",http://app.strava.com/segments/,None
9666,Allen Clark Hill Climb,2017-10-01 00:00:00,"Waitsfield, VT",downloadrace.php?raceID=9666&json=1,"Clear, 56 degrees, wind 5 mph",http://app.strava.com/segments/16186267,None


## Download JSON results

In [150]:
urls = df['json_url']

In [151]:
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/{url}') for url in urls]

In [159]:
# takes a few minutes (depending on caching)
for (index, url), future in zip(urls.items(), futures):
    dill.dump(future.result().text, open(f'C:/data/results/races/{index}.pkd', 'wb'))

## Load a sample race into a dataframe

In [160]:
import glob
import re
import dill
import pandas as pd

In [163]:
file = r'C:\data\results\races\1000.pkd'
json = dill.load(open(file, 'rb'))

df_race = pd.read_json(json)

# drop duplicate columns
df_race = df_race.drop(columns=columns)

race_id = re.findall('(\d+)', file)[0]
df_race['race_id'] = int(race_id)


In [164]:
df_race.head()

,RaceName,RaceCategoryName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,...,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID,race_id
0,Swan Island Rose Festival Criterium,Category 3,1,12069,Luke,Demoe,SLOCUM Race Team,,255.000000,300.0,...,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,49081,1000
1,Swan Island Rose Festival Criterium,Category 3,2,14499,Joe,Dengel,,,258.333333,300.0,...,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,49082,1000
2,Swan Island Rose Festival Criterium,Category 3,3,14953,Daniel,Penner,Ironclad Performance Wear,,261.666667,300.0,...,3.0,NaN,NaN,25.0,31,NaN,199994.0,NaN,49083,1000
3,Swan Island Rose Festival Criterium,Category 3,4,10755,Chris,Ward,Guinness Cycling Team,,265.000000,300.0,...,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,49084,1000
4,Swan Island Rose Festival Criterium,Category 3,5,11409,Brian,Engelhard,bicycleattorney.com,,268.333333,300.0,...,2.0,NaN,NaN,36.0,31,NaN,358937.0,NaN,49085,1000


## Load all races

In [3]:
import glob
import re
import dill
import pandas as pd

In [4]:
df = pd.read_pickle('C:/data/results/df.pkl')
df = df[df['json_url'] != '']
df = df.shift(1).drop(index=1)  # index was shifted for some reason
df['weather'] = df['weather'].str.split('\n').str.get(-1)  # sometimes the weather column also got "Series: " information
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
4623,CRCA Racing Series 2014,2014-05-24 00:00:00,"New York, NY",downloadrace.php?raceID=4623&json=1,"Clear, 63 degrees, wind 8 mph",http://app.strava.com/segments/,None
2763,Plainville Spring Series,2013-03-30 00:00:00,"Plainville, CT",downloadrace.php?raceID=2763&json=1,"Clear, 52 degrees, wind 11 mph",http://app.strava.com/segments/,None
9168,NSGP - North Mankato Road Race,2017-06-17 00:00:00,"North Mankato, MN",downloadrace.php?raceID=9168&json=1,"Mostly Cloudy, 77 degrees, wind 10 mph",http://app.strava.com/segments/,None
10673,2018 USA Cycling Amateur Road Nationals - Crit,2018-07-29 00:00:00,"Hagerstown, MD",downloadrace.php?raceID=10673&json=1,"Clear, 77 degrees, wind 3 mph",http://app.strava.com/segments/,None
8457,Valley of the Sun Stage Race - Crit,2017-02-19 00:00:00,"Phoenix, AZ",downloadrace.php?raceID=8457&json=1,"Clear, 53 degrees, wind 7 mph",http://app.strava.com/segments/,None


In [48]:
columns = [str(i) for i in range(28)]  # Each column repeated for some reason
for file in glob.iglob(r'C:\data\results\races\*.pkd'):    
    json = dill.load(open(file, 'rb'))

    df_race = pd.read_json(json)
    if df_race.empty:
        continue
    
    # drop duplicate columns
    df_race = df_race.drop(columns=columns)

    race_id = re.findall('(\d+)', file)[0]
    df_race['race_id'] = int(race_id) + 2 # somehow offset by 2
    raise Exception('Load just one for now!')

Exception: Load just one for now!

In [72]:
df_race.head(5)

,RaceName,RaceCategoryName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,...,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID,race_id
0,NCC 10th Tour of the Hilltowns,Cat 4,1,699,Matt,Cuttler,CRCA/Affinity Cycles,2:28:53,340.000000,400.000000,...,3.0,NaN,NaN,29.0,94,NaN,266432,NaN,504,2
1,NCC 10th Tour of the Hilltowns,Cat 4,2,700,Eric,Weinrich,portland velo-cycle-mania,st,341.276596,446.250000,...,3.0,NaN,NaN,42.0,94,NaN,227867,NaN,505,2
2,NCC 10th Tour of the Hilltowns,Cat 4,3,701,Michael,Boardman,,st,342.553191,346.428571,...,3.0,NaN,NaN,34.0,94,NaN,241308,NaN,506,2
3,NCC 10th Tour of the Hilltowns,Cat 4,4,702,John,Nobile,,st,343.829787,400.000000,...,4.0,NaN,NaN,45.0,94,NaN,26169,NaN,507,2
4,NCC 10th Tour of the Hilltowns,Cat 4,5,703,Ryan,Short,TEAM PLACID PLANET,st,345.106383,400.000000,...,4.0,NaN,NaN,20.0,94,NaN,0217099,NaN,508,2


In [73]:
df_race.columns

Index(['RaceName', 'RaceCategoryName', 'Place', 'RacerID', 'FirstName',
       'LastName', 'TeamName', 'RaceTime', 'Points', 'PriorPoints',
       'RaceCategoryID', 'IsDnf', 'IsDQ', 'IsDNP', 'OffTheFront', 'OffTheBack',
       'GroupSprintPlace', 'FieldSprintPlace', 'TeamID', 'Category',
       'Starters', 'ReportedAge', 'CalculatedAge', 'RacerCount',
       'FinishPhotoUrl', 'License', 'MetaDataUrl', 'ResultID', 'race_id'],
      dtype='object')

In [57]:
df_race[['RaceCategoryName', 'RaceCategoryID']]

,RaceCategoryName,RaceCategoryID
0,Cat 4,19
1,Cat 4,19
2,Cat 4,19
3,Cat 4,19
4,Cat 4,19
...,...,...
432,Women Pro/1/2/3,15
433,Women Pro/1/2/3,15
434,Women Pro/1/2/3,15
435,Women Pro/1/2/3,15


In [58]:
df_race.groupby(['RaceCategoryID', 'RaceCategoryName']).count()

,,RaceName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,IsDnf,...,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID,race_id
RaceCategoryID,RaceCategoryName,,,,,,,,,,,,,,,,,,,,,
13,cat 5,43,43,43,43,43,43,43,43,43,43,...,33,0,0,33,43,0,33,0,43,43
14,Pro 1/2/3,48,48,48,48,48,48,48,48,48,48,...,43,0,0,43,48,0,44,0,48,48
15,Women Pro/1/2/3,25,25,25,25,25,25,25,25,25,25,...,22,0,0,22,25,0,22,0,25,25
16,Men 40+ (Cat 1-,63,63,63,63,63,63,63,63,63,63,...,53,0,0,53,63,0,55,0,63,63
17,Men Cat 3,75,75,75,75,75,75,75,75,75,75,...,61,0,0,61,75,0,65,0,75,75
18,Women Cat 3/4,20,20,20,20,20,20,20,20,20,20,...,15,0,0,15,20,0,15,0,20,20
19,Cat 4,94,94,94,94,94,94,94,94,94,94,...,81,0,0,81,94,0,82,0,94,94
20,Men 50+ (Cat 1-,69,69,69,69,69,69,69,69,69,69,...,62,0,0,62,69,0,63,0,69,69


In [61]:
fix_same_time(df_race[df_race['RaceCategoryID']==19]['RaceTime'])

In [63]:
racetime = df_race[df_race['RaceCategoryID']==19]['RaceTime']

True

In [47]:
i = 28
df_race.columns[i], df_race.iloc[:,i].unique()

('race_id', array([2], dtype=int64))

In [ ]:
cols = ['']

In [11]:
df_race['RaceTime'] = pd.to_datetime(df_race['RaceTime'], errors='coerce')
df_race['RaceTime']

0     2020-10-02 02:28:53
1                     NaT
2                     NaT
3                     NaT
4                     NaT
              ...        
432   2020-10-02 00:17:36
433   2020-10-02 00:20:27
434   2020-10-02 00:21:53
435                   NaT
436   2020-10-02 00:35:22
Name: RaceTime, Length: 437, dtype: datetime64[ns]